In [104]:
# Adapted from:
# https://stackoverflow.com/questions/492519/timeout-on-a-function-call
import threading
import _thread as thread
import sys

import time

from datetime import datetime as dt

from importlib import reload
import src.student
reload(src.student)

from src.student import StudentFunction

In [105]:
def inf_loop():
    x = 1
    while True:
        x+=1
    return f'hello {x}'

def inf_loop_handle():
    x = 1
    try:
        while True:
            x+=1
    except:
        print('it caught something')
    return f'hello {x}'

def it_sleeps():
    time.sleep(5)
    return 'I slept'

def fun_with(a,b,c):
    return a - b + c

In [106]:
sf = StudentFunction(it_sleeps, 
                     timeout_secs=1
                    )

In [107]:
sf.run_fun()

Interrupt sent when it_sleeps ran more than 1 secs.


In [108]:
sf.result

'Error: Timed out after 1 secs.'

In [109]:
sf.start_time, sf.stop_time, sf.runtime

(datetime.datetime(2025, 2, 2, 17, 14, 51, 642899),
 datetime.datetime(2025, 2, 2, 17, 14, 56, 647196),
 datetime.timedelta(seconds=5, microseconds=4297))

In [110]:
sf.clean_exit

nan

In [111]:
sf.interrupted

True

In [112]:
x = ['a', 'b']
s = 'a'
s in x

True

In [88]:
def quit_function(fn_name):
    # print to stderr, unbuffered in Python 2.
    
    print('{0} took too long'.format(fn_name), file=sys.stderr)
    sys.stderr.flush() # Python 3 stderr is likely buffered.
    thread.interrupt_main() # raises KeyboardInterrupt
    return

def exit_after(s):
    '''
    use as decorator to exit process if 
    function takes longer than s seconds
    '''
    def outer(fn):
        def inner(*args, **kwargs):
            timer = threading.Timer(s, quit_function, args=[fn.__name__])
            #timer = threading.Timer(s, thread.interrupt_main)#, args=[fn.__name__])
            timer.start()
            try:
                result = fn(*args, **kwargs)
            except KeyboardInterrupt as e:
                result = 'Timed Out'
            except Exception as e:
                result = e
            finally:
                timer.cancel()
            return result
        return inner
    return outer

In [89]:
@exit_after(1)
def it_sleeps():
    time.sleep(3)
    return 'I slept'

@exit_after(1)
def inf_loop():
    x = 1
    while True:
        x+=1
    return f'hello {x}'

# Internal error handling breaks it...
# but we can probably solve with some logging
@exit_after(1)
def inf_loop_handle():
    x = 1
    try:
        while True:
            x+=1
    except:
        print('it caught something')
    return f'hello {x}'

In [90]:
funcs = [it_sleeps, inf_loop, inf_loop_handle]
results = []
for fun in funcs:
    try:
        results.append(fun())
    except KeyboardInterrupt as e:
        print('caught out here')
        print(type(e))

it_sleeps took too long
inf_loop took too long
inf_loop_handle took too long


it caught something


In [91]:
results

['Timed Out', 'Timed Out', 'hello 32931409']

In [102]:
def fun(*args, x = None, **kwargs):
    print(args)
    print(x)
    print(kwargs)
    return

In [104]:
fun(1,2,3,cat=10)

(1, 2, 3)
None
{'cat': 10}
